## Imports

In [478]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Read and Analyze data

In [370]:
# Train Data

training = pd.read_excel("../datasets/training_set.xlsx")
print(training.shape)
training.head()

(575, 3)


,title,cleaned text,relevance
0,Blackstone to buy EagleClaw Midstream for abou...,"EagleClaw Midstream Ventures LLC, the largest ...",0
1,Worldwide Mobile Crusher and Screener Industry...,WireThe report has been added to offering. Acc...,0
2,"In a First, BP Offers Employees Shares in Rall...",Oil major BP is launching its first share awar...,1
3,SHAREHOLDER ALERT: Purcell Julie & Lefkowitz L...,TipRanks We’ve got a full month of 2021 behind...,0
4,Komatsu Australia - Komatsu Australia,Komatsu Australia Corporate Finance Pty Ltd Cr...,0


In [371]:
# Test Data

testing = pd.read_excel("../datasets/testing_set.xlsx")
print(testing.shape)
testing.head()

(247, 3)


,title,cleaned text,relevance
0,As Blackjewel bankruptcy case enters final str...,On June 11 the court ruled it would lift the a...,0
1,FG Wilson (Engineering) Ltd v John Holt & Comp...,1. This is the hearing of an application by th...,0
2,Personnel Policy and Performance Appraisal Sys...,Here at Komatsu we consider many aspects of ou...,0
3,Caterpillar planning 700 job cuts in the North,US manufacturing firm Caterpillar has announce...,1
4,Hardman & Co Research : Tritax EuroBox present...,Hardman & Co Research 03-Feb-2021 / 12:45 GMT/...,0


In [372]:
training.isnull().sum()

title           0
cleaned text    0
relevance       0
dtype: int64

In [373]:
testing.isnull().sum()

title           0
cleaned text    0
relevance       0
dtype: int64

## Data cleaning

1. Remove stopwords
2. Remove special characters
2. Lemmatization
3. Embedding

In [374]:
import spacy

nlp = spacy.load('en_core_web_sm')

# Text Preprocessing with varoius combination

def spacy_process(text, remove_stopwords, remove_punctuation):
    
    doc = nlp(text)
    
    # Lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
  
    # Filter the stopword

    if remove_stopwords:
        filtered_sentence =[] 
        for word in lemma_list:
            lexeme = nlp.vocab[word]
            if lexeme.is_stop == False:
                filtered_sentence.append(word)
    else:
        filtered_sentence =  lemma_list
    
    # Remove punctuation
    if remove_punctuation:
        punctuations="?:!.,;$\'-_"
        for word in filtered_sentence:
            if word in punctuations:
                filtered_sentence.remove(word)

    return " ".join(filtered_sentence)

In [375]:
# Converting Text to lowercase

def lower_text(text):
    return text.strip().lower()

In [376]:
training.iloc[:,1] = training.iloc[:,1].apply(spacy_process, args=(True, True))
training = training.iloc[:,1:]

In [377]:
testing.iloc[:,1] = testing.iloc[:,1].apply(spacy_process, args=(True, True))
testing = testing.iloc[:,1:]

## Initialize Hyperparameters

In [563]:
VOCAB_SIZE = 500
OUTPUT_LENGTH = 128
OPTIMIZER = Adam()
TEST_SIZE = 0.2
NUM_EPOCHS = 500
BATCH_SIZE = 512

## Prepare training data

In [564]:
tokenizer = Tokenizer(
    num_words=VOCAB_SIZE,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True
)

# Convert Text to list of tokens
tokenizer.fit_on_texts(training.iloc[:,0])

# Create NumPy array of training data
X = tokenizer.texts_to_sequences(training.iloc[:,0])

# Pad all sequences to the same length
X = pad_sequences(X)

In [565]:
y = training.iloc[:,1]

In [566]:
model = Sequential()
model.add(Embedding(
    VOCAB_SIZE, 
    OUTPUT_LENGTH,
    input_length = X.shape[1]
))

# Add LSTM layers
model.add(GRU(100)) # Just specify num neurons if you're using Embedding
model.add(Dense(250))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer=OPTIMIZER, metrics = ['accuracy'])
print(model.summary())

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_42 (Embedding)    (None, 2525, 128)         64000     
                                                                 
 gru_38 (GRU)                (None, 100)               69000     
                                                                 
 dense_49 (Dense)            (None, 250)               25250     
                                                                 
 dropout_7 (Dropout)         (None, 250)               0         
                                                                 
 dense_50 (Dense)            (None, 1)                 251       
                                                                 
Total params: 158,501
Trainable params: 158,501
Non-trainable params: 0
_________________________________________________________________
None


In [567]:
X_train = training.iloc[:,0]
y_train = training.iloc[:,1]

In [568]:
X_valid = testing.iloc[:,0]
y_valid = testing.iloc[:,1]

In [569]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [570]:
y_train.unique()

array([0, 1], dtype=int64)

## Model training and evaluation

In [571]:
model.fit(
    X_train,
    y_train,
    epochs=NUM_EPOCHS, 
    batch_size=BATCH_SIZE
)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 0.6933 - accuracy: 0.4826
Epoch 2/500
1/1 [==============================] - 0s 366ms/step - loss: 0.6837 - accuracy: 0.6935
Epoch 3/500
1/1 [==============================] - 0s 377ms/step - loss: 0.6752 - accuracy: 0.6870
Epoch 4/500
1/1 [==============================] - 0s 355ms/step - loss: 0.6643 - accuracy: 0.6891
Epoch 5/500
1/1 [==============================] - 0s 384ms/step - loss: 0.6514 - accuracy: 0.6891
Epoch 6/500
1/1 [==============================] - 0s 368ms/step - loss: 0.6379 - accuracy: 0.6891
Epoch 7/500
1/1 [==============================] - 0s 361ms/step - loss: 0.6231 - accuracy: 0.6891
Epoch 8/500
1/1 [==============================] - 0s 381ms/step - loss: 0.6043 - accuracy: 0.6891
Epoch 9/500
1/1 [==============================] - 0s 365ms/step - loss: 0.5875 - accuracy: 0.6891
Epoch 10/500
1/1 [==============================] - 0s 369ms/step - loss: 0.5673 - accuracy: 0.6891
Epoch 11/500

In [572]:
y_pred = model.predict(X_valid)

4/4 [==============================] - 1s 73ms/step


In [573]:
y_pred

array([[1.00000000e+00],
       [7.10985437e-10],
       [1.09314788e-08],
       [5.20668795e-12],
       [3.81305298e-08],
       [9.99985456e-01],
       [6.52013523e-06],
       [1.36316777e-10],
       [9.87797193e-07],
       [6.75641167e-15],
       [3.61599010e-12],
       [1.00000000e+00],
       [7.96494811e-15],
       [6.00974559e-08],
       [1.71570064e-04],
       [1.42334189e-09],
       [6.97492389e-03],
       [3.41587036e-10],
       [1.33442892e-08],
       [4.91811484e-02],
       [5.20668795e-12],
       [1.58033162e-01],
       [3.57418685e-05],
       [1.50029003e-08],
       [2.49566824e-07],
       [9.99806225e-01],
       [9.99998212e-01],
       [1.02504205e-09],
       [1.00000000e+00],
       [1.00000000e+00],
       [2.34734699e-07],
       [2.39640165e-11],
       [1.03554336e-17],
       [2.15643969e-09],
       [3.05994063e-10],
       [1.01139236e-07],
       [6.00974559e-08],
       [9.48469460e-06],
       [1.00000000e+00],
       [9.44652986e-15],


In [574]:
y_pred = np.where(y_pred<0.5, 0, 1)
y_pred


array([[1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
    

In [575]:
print(accuracy_score(y_valid, y_pred))

0.7652173913043478
